## Set things up

In [1]:
%matplotlib inline
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor  # not sure I'm going to use this
from sklearn.model_selection import GridSearchCV

In [2]:
RAW_DATA_SOURCE = "C:/Users/sbranchett/Data/WiFi_data/WiFiAccessPoint.csv"
CLEAN_DATA_SOURCE = "CleanedWiFiAccessPoint.csv"

## Read in data

In [3]:
def load_wifi_data(path):
    """
    Read WiFi clientCount .csv file and sum the clientCounts over building and timestamp, bucketed to 5 minutes
    
    Input: filepath the .csv file
    Output: Dataframe with "building", "time_bucket", "clientCount"
    Columns "building" generated from "locationHierarchy", and "time_bucket" generated from "timestamp"
    """
    all_data = pd.read_csv(path, delimiter=",")
    all_data = all_data.rename(columns=lambda x: x.strip())  # get rid of extra spaces in column names

    sum_clientCounts = all_data["clientCount"].sum()
    
    # extract building from 'locationHierarchy' string and deal with unknown buildings
    all_data["building"] = all_data["locationHierarchy"].str.split(" > ",expand=True)[1]
    all_data["building"] = all_data["building"].fillna("Unknown")

    # convert timestamp from epoch milliseconds to 5 minute buckets (1000 milliseconds * 60 seconds * 5 minutes)
    all_data["time_bucket"] = all_data["timestamp"].apply(lambda d: 300000*int(d/300000))
    
    # keep only the interesting columns
    all_data = all_data[["time_bucket", "building", "clientCount"]]
    
    # sum data over buildings and time buckets
    all_data = all_data.groupby(["time_bucket", "building"]).sum()  # sort on time first so that new data keeps sequence
    
    assert sum_clientCounts == all_data["clientCount"].sum()  # check that no clientCounts went missing

    return all_data

# if possible read in the cleaned data, otherwise clean the raw data and save
if os.path.isfile(CLEAN_DATA_SOURCE):
    all_data = pd.read_csv(CLEAN_DATA_SOURCE, delimiter=",")
else:
    all_data = load_wifi_data(RAW_DATA_SOURCE)
    all_data.to_csv(CLEAN_DATA_SOURCE)

print(all_data["clientCount"].sum())  # should be 3597531

3597531


In [4]:
all_data = all_data.reset_index().drop("index", axis=1)  # get rid of multiindex for StratifiedShuffleSplit
print(all_data)

         time_bucket               building  clientCount
0      1620991800000      03-Science Center           34
1      1620991800000             05-TNW-BIO            1
2      1620991800000             08-BK-City            2
3      1620991800000   19-Studuitzendbureau           13
4      1620991800000                20-Aula           32
...              ...                    ...          ...
78161  1621608600000                 64-HSL           10
78162  1621608600000          66-Fellowship          126
78163  1621608600000  Katalyse Labaratorium            8
78164  1621608600000                Unknown           32
78165  1621608600000           VLL-LAB(TNO)           11

[78166 rows x 3 columns]


## Separate Test Set stratified over buildings

In [5]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(all_data, all_data["building"]):
    strat_train_set = all_data.loc[train_index]
    strat_test_set = all_data.loc[test_index]

print(strat_train_set)

         time_bucket                           building  clientCount
11688  1621083900000                    37-Sportcentrum           15
61351  1621476000000                         36-ESP-Lab            1
55780  1621431900000                      66-Fellowship          101
26158  1621198200000                          32-OCP-IO           12
34879  1621266900000                             64-HSL           16
...              ...                                ...          ...
50892  1621393500000                            28- WNI            4
33498  1621256100000  36-EWI LB_K t/m 3 & HB_K  t/m 2e           247
4160   1621024500000                      35-Drebbelweg            2
56082  1621434300000                          63-Simona            8
55786  1621432200000                         08-BK-City          670

[62532 rows x 3 columns]


## Create Work days and Student days

In [6]:
uni_hols = (datetime.date(year=2021, month=5, day=5),
            datetime.date(year=2021, month=5, day=13),
            datetime.date(year=2021, month=5, day=14),
            datetime.date(year=2021, month=5, day=24)
           )  # National Holidays and Collective Free days

def roster(date):
    # categorise the days of the academic year at TU Delft between 1 May 2021 and 29 August 2021
    
    if (date > datetime.date(year=2021, month=5, day=16)) and \
       (date < datetime.date(year=2021, month=5, day=22)):
        categorie = "Exam_BSc"
    elif (date > datetime.date(year=2021, month=6, day=15)) and \
         (date < datetime.date(year=2021, month=6, day=19)):
        categorie = "Study_mixed"
    elif (date > datetime.date(year=2021, month=6, day=22)) and \
         (date < datetime.date(year=2021, month=6, day=26)):
        categorie = "Exam_mixed"
    elif (date > datetime.date(year=2021, month=6, day=29)) and \
         (date < datetime.date(year=2021, month=7, day=3)):
        categorie = "Exam"
    elif (date > datetime.date(year=2021, month=8, day=8)) and \
         (date < datetime.date(year=2021, month=8, day=14)):
        categorie = "Exam"
    elif (date > datetime.date(year=2021, month=7, day=4)):
        categorie = "Free"
    else:
        categorie = "Learn"
    return categorie

## Separate predictions and labels

In [7]:
crowd = strat_train_set.drop("clientCount", axis=1)
crowd_labels = strat_train_set["clientCount"].copy()
print(crowd)

         time_bucket                           building
11688  1621083900000                    37-Sportcentrum
61351  1621476000000                         36-ESP-Lab
55780  1621431900000                      66-Fellowship
26158  1621198200000                          32-OCP-IO
34879  1621266900000                             64-HSL
...              ...                                ...
50892  1621393500000                            28- WNI
33498  1621256100000  36-EWI LB_K t/m 3 & HB_K  t/m 2e 
4160   1621024500000                      35-Drebbelweg
56082  1621434300000                          63-Simona
55786  1621432200000                         08-BK-City

[62532 rows x 2 columns]


In [8]:
assert crowd.isnull().values.any() == False  # check there are no missing values

## Build Transformers

In [9]:
# specify which extra attributes to add to the data
# want to treat these as hyperparameters for grid search
add_day_of_week=False
add_time_of_day=False
add_uni_hols=False
add_weekend=False
add_academic_yr=False

In [10]:
# Set up categories for OneHotEncoder.
# This is needed later for the transformation of small sets that do not contain all the categories.

buildings = list(set(all_data["building"].values))
rosters = ["Exam_BSc", "Study_mixed", "Exam_mixed", "Exam", "Free", "Learn"]  # get a list of all roster categories, SEB could be improved

In [11]:
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_day_of_week=True, add_time_of_day=True, add_uni_hols=True, add_weekend=True, add_academic_yr=True):
        self.add_day_of_week = add_day_of_week
        self.add_time_of_day = add_time_of_day
        self.add_uni_hols = add_uni_hols
        self.add_weekend = add_weekend
        self.add_academic_yr = add_academic_yr

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_trans = X.copy()
        if self.add_day_of_week:
            X_trans["day_of_week"] = X["time_bucket"].apply(lambda d: datetime.datetime.fromtimestamp(d/1000).weekday())
        if self.add_time_of_day:
            X_trans["time_of_day"] = X["time_bucket"].apply(lambda d: int(d%86400000))
        if self.add_uni_hols:
            X_trans["hols"] = X["time_bucket"].apply(lambda d: 1. if datetime.datetime.fromtimestamp(d/1000).date() in uni_hols else 0.)
        if self.add_weekend:
            X_trans["weekend"] = X["time_bucket"].apply(lambda d: 1. if datetime.datetime.fromtimestamp(d/1000).weekday() > 4 else 0.)
        if self.add_academic_yr:
            X_trans["academic_yr"] = X["time_bucket"].apply(lambda d: roster(datetime.datetime.fromtimestamp(d/1000).date()))
        return X_trans

In [12]:
# Moved the column transformation to it's own class so that it could be dependent on attributes
# Pity I have to specifiy the same attributes to both classes
class CombinedColumnTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, add_day_of_week=True, add_time_of_day=True, add_uni_hols=True, add_weekend=True, add_academic_yr=True,
                 buildings=None, rosters=None):
        self.add_day_of_week = add_day_of_week
        self.add_time_of_day = add_time_of_day
        self.add_uni_hols = add_uni_hols
        self.add_weekend = add_weekend
        self.add_academic_yr = add_academic_yr
        self.buildings = buildings
        self.rosters = rosters
        self.num_attribs = ["time_bucket"]
        self.cat_attribs = ["building"]
        self.all_categories = [buildings]
        if self.add_day_of_week:
            self.num_attribs.extend(["day_of_week"])
        if self.add_time_of_day:
            self.num_attribs.extend(["time_of_day"])
        if self.add_uni_hols:
            self.num_attribs.extend(["hols"])
        if self.add_weekend:
            self.num_attribs.extend(["weekend"])
        if self.add_academic_yr:
            self.all_categories.append(rosters)
            self.cat_attribs.extend(["academic_yr"])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        coltrans = ColumnTransformer([("num", StandardScaler(), self.num_attribs),
                                      ("cat", OneHotEncoder(categories=self.all_categories), self.cat_attribs)
                                     ])
        X_trans = coltrans.fit_transform(X)
        print('SEB',X_trans.shape, self.add_day_of_week)
        return X_trans

# combine the pipelines based on column type

In [13]:
full_pipeline = Pipeline([
    ('add_attributes', CombinedAttributesAdder(add_day_of_week, add_time_of_day, add_uni_hols, add_weekend,
                                               add_academic_yr)),
    ('trans_columns', CombinedColumnTransformer(add_day_of_week, add_time_of_day, add_uni_hols, add_weekend,
                                                add_academic_yr, buildings=buildings, rosters=rosters)),
    ('lin_reg',  LinearRegression()),
])

In [14]:
print(full_pipeline.named_steps['add_attributes'])

CombinedAttributesAdder(add_academic_yr=False, add_day_of_week=False,
                        add_time_of_day=False, add_uni_hols=False,
                        add_weekend=False)


In [15]:
print(full_pipeline.named_steps['trans_columns'])

CombinedColumnTransformer(add_academic_yr=False, add_day_of_week=False,
                          add_time_of_day=False, add_uni_hols=False,
                          add_weekend=False,
                          buildings=['08-BK-City', 'Unknown', '43-EGM',
                                     '50-TNW-RID', '03-Science Center',
                                     '25-GreenVillage', '21-BTUD',
                                     '36-EWI LB_K t/m 3 & HB_K  t/m 2e ',
                                     '28- WNI', '31-TBM', '33-Pulse', '45-LSL',
                                     '37-Sportcentrum', '60-LMS',
                                     '26-Bouwcampus', '34-3ME', '66-Fellowship',
                                     '35-Drebbelweg', '61-Vliegtuighal',
                                     '64-HSL', '05-TNW-BIO',
                                     '38-Cultureel Centrum', '23-CITG',
                                     '22-TNW-TN', '63-Simona',
                                 

In [16]:
crowd_fitted = full_pipeline.fit(crowd, crowd_labels)
print(crowd_fitted)

SEB (62532, 39) False
Pipeline(steps=[('add_attributes',
                 CombinedAttributesAdder(add_academic_yr=False,
                                         add_day_of_week=False,
                                         add_time_of_day=False,
                                         add_uni_hols=False,
                                         add_weekend=False)),
                ('trans_columns',
                 CombinedColumnTransformer(add_academic_yr=False,
                                           add_day_of_week=False,
                                           add_time_of_day=False,
                                           add_uni_hols=False,
                                           add_weekend=False,
                                           buildings=['08-BK-City', 'Unknown',
                                                      '43-EGM...
                                                      '37-Sportcentrum',
                                                      

In [17]:
param_grid = {'add_attributes__add_day_of_week': [True, False], 'add_attributes__add_time_of_day': [True, False],
              'add_attributes__add_uni_hols': [True, False],
              'add_attributes__add_weekend': [True, False], 'add_attributes__add_academic_yr': [True, False],
              'trans_columns__add_day_of_week': ['add_attributes__add_day_of_week'],
              'trans_columns__add_time_of_day': ['add_attributes__add_time_of_day'],
              'trans_columns__add_uni_hols': ['add_attributes__add_uni_hols'],
              'trans_columns__add_weekend': ['add_attributes__add_weekend'],
              'trans_columns__add_academic_yr': ['add_attributes__add_academic_yr'],
              'trans_columns__buildings': [buildings], 'trans_columns__rosters': [rosters]}


grid_search = GridSearchCV(full_pipeline, param_grid, scoring='neg_mean_squared_error', return_train_score=True)

In [18]:
grid_search.fit(crowd, crowd_labels)

SEB (50025, 39) add_attributes__add_day_of_week
SEB (12507, 39) add_attributes__add_day_of_week
SEB (50025, 39) add_attributes__add_day_of_week
SEB (50025, 39) add_attributes__add_day_of_week
SEB (12507, 39) add_attributes__add_day_of_week
SEB (50025, 39) add_attributes__add_day_of_week
SEB (50026, 39) add_attributes__add_day_of_week
SEB (12506, 39) add_attributes__add_day_of_week
SEB (50026, 39) add_attributes__add_day_of_week
SEB (50026, 39) add_attributes__add_day_of_week
SEB (12506, 39) add_attributes__add_day_of_week
SEB (50026, 39) add_attributes__add_day_of_week
SEB (50026, 39) add_attributes__add_day_of_week
SEB (12506, 39) add_attributes__add_day_of_week
SEB (50026, 39) add_attributes__add_day_of_week
SEB (50025, 39) add_attributes__add_day_of_week
SEB (12507, 39) add_attributes__add_day_of_week
SEB (50025, 39) add_attributes__add_day_of_week
SEB (50025, 39) add_attributes__add_day_of_week
SEB (12507, 39) add_attributes__add_day_of_week
SEB (50025, 39) add_attributes__add_day_

KeyboardInterrupt: 

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params['add_attributes__add_academic_yr'], params['trans_columns__add_academic_yr'])

In [ ]:
bugadebug

## Select and Train a model

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(crowd_prepared, crowd_labels)

In [ ]:
# crowd_attributes = CombinedAttributesAdder(add_day_of_week, add_time_of_day, add_uni_hols, add_weekend, add_academic_yr)

In [ ]:
# prepare a small data set to test
some_data = crowd.iloc[:5]
some_labels = crowd_labels[:5]

some_data_extra_attribs = crowd_attributes.transform(some_data)
print(some_data_extra_attribs)

some_data_prepared = full_pipeline.transform(some_data_extra_attribs)  # this gives a sparse matrix!
print(some_data_prepared)
print(some_data_prepared.shape)

In [ ]:
# check out the predictions
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

In [ ]:
crowd_predictions = lin_reg.predict(crowd_prepared)
lin_mse = mean_squared_error(crowd_labels, crowd_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

In [ ]:
lin_scores = cross_val_score(lin_reg, crowd_prepared, crowd_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

In [ ]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(crowd_prepared, crowd_labels)

crowd_predictions = tree_reg.predict(crowd_prepared)
tree_mse = mean_squared_error(crowd_labels, crowd_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

In [ ]:
scores = cross_val_score(tree_reg, crowd_prepared, crowd_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

display_scores(tree_rmse_scores)

In [ ]:
# check out the predictions
print("Predictions:", tree_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

## Let's take a look at how are predictions are doing

In [ ]:
first_time_stamp = all_data["time_bucket"].min()
last_time_stamp = all_data["time_bucket"].max()

a_building = "20-Aula"
a_building_data = strat_train_set.loc[strat_train_set["building"] == a_building]

In [ ]:
times = list(range(first_time_stamp, last_time_stamp + 300000, 300000))

test = pd.DataFrame()
test["time_bucket"] = times
test["building"] = a_building

test_extra_attribs = crowd_attributes.transform(test)
test_prepared = full_pipeline.transform(test_extra_attribs)

lin_preds = lin_reg.predict(test_prepared)
tree_preds = tree_reg.predict(test_prepared)
xes = test["time_bucket"]

plt.figure(figsize=(20,5))
plt.title(a_building)
plt.axis([first_time_stamp, last_time_stamp, -100, 200])
plt.plot(a_building_data["time_bucket"], a_building_data["clientCount"], 'bo', times, lin_preds, 'g--', times, tree_preds, 'r--')
plt.show()

## See what happens when we try to predict the future

In [ ]:
extra_days = 1
latest_time_stamp = last_time_stamp + extra_days*86400000 + 300000
times = list(range(first_time_stamp, latest_time_stamp, 300000))

test = pd.DataFrame()
test["time_bucket"] = times
test["building"] = a_building
test_extra_attribs = crowd_attributes.transform(test)
test_prepared = full_pipeline.transform(test_extra_attribs)
lin_preds = lin_reg.predict(test_prepared)
tree_preds = tree_reg.predict(test_prepared)

plt.figure(figsize=(20,5))
plt.title(a_building)
plt.axis([first_time_stamp, latest_time_stamp, -100, 200])
plt.plot(a_building_data["time_bucket"], a_building_data["clientCount"], 'bo', times, lin_preds, 'g--', times, tree_preds, 'r--')
plt.show()

Save what we have so far:
```
from joblib import dump

dump(lin_reg, "lin_reg.joblib")
dump(tree_reg, "tree_reg.joblib")
dump(forest_reg, "forest_reg.joblib")
```

Still want to do:

* add grid search
* possibly add time from lunch as an attribute
* see what happens with 4 weeks of data
* write blog
* illustrate blog with nice graphs

In [ ]:
prepare_select_and_predict_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('feature_selection', TopFeatureSelector(feature_importances, k)),
    ('svm_reg', SVR(**rnd_search.best_params_))
])

prepare_select_and_predict_pipeline.fit(housing, housing_labels)

In [ ]:
param_grid = [
    {'add_day_of_week': [True, False], 'add_time_of_day': [True, False], 'add_uni_hols': [True, False],
     'add_weekend': [True, False], 'add_academic_yr': [True, False]}
]

# lin_reg = LinearRegression()

grid_search = GridSearchCV(lin_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(crowd_prepared, crowd_labels)

See also: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html